In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import random
import re
import urllib.request
import os

# Edge WebDriver 的路徑，根據你的系統和 WebDriver 的位置進行修改
def opebwebdriver(edge_driver_path = './msedgedriver.exe'):
    headers = {
        'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1'
    }

    # 創建 Edge 瀏覽器實例
    options = webdriver.EdgeOptions()
    
    for key, value in headers.items():
        options.add_argument(f"{key}={value}")
    options.add_argument("--mute-audio")  # 設定瀏覽器静音
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--disable-extensions")
    # 啟動 Edge 瀏覽器
    driver = webdriver.Edge(service=EdgeService(executable_path=edge_driver_path), options=options)
    driver.maximize_window()
    driver.set_script_timeout(180)
    return driver

def login(driver, username, password):
    url = 'https://www.instagram.com/accounts/login/'
    driver.get(url)
    while True:
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//input[contains(@name, 'username')]"))
            ).send_keys(username)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//input[contains(@name, 'password')]"))
            ).send_keys(password)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//button[contains(@type, 'submit')]"))
            ).click()
        except Exception as e:
            driver.refresh()
        else:
            break
            
def find_all_post_link(driver, username):
    url = 'https://www.instagram.com/'+username
    driver.get(url)
    # 下滑頁面載入後續資料
    links = []
    curr_link_num = -1
    while True:
        if curr_link_num==len(links):
            break
        curr_link_num = len(links)
#     for i in range(100):
        try:
            post = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located(((By.XPATH, "//a[contains(@href, '/p/')]")))
            )
        except Exception as e:
            driver.refresh()
            continue
        else:
            for href in post:
                if href.get_attribute('href') not in links:
                    links.append(href.get_attribute("href"))
            
        # 模擬下滑，滾動到頁面底部
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         do_sleep()
        time.sleep(4)
    return links

def remove_special_characters(input_string):
    # 使用正則表達式替換特殊符號
    cleaned_string = re.sub(r'[^a-zA-Z0-9\s]', '', input_string)
    return cleaned_string

def find_images_videos(driver):
    # find images and click next button to next image
    image_links = []
    video_links = []
    while True:
        try:
            # do_sleep()
            videos = WebDriverWait(driver, 1).until(
                EC.presence_of_all_elements_located(((By.XPATH, '//video')))
            )
        except Exception as e:
            # do_sleep()
            try:
                images = WebDriverWait(driver, 1).until(
                    EC.presence_of_all_elements_located(((By.XPATH, '//div/div[@class="_aagu"]/div[@class="_aagv"]')))
                )
            except Exception as e:
                driver.refresh()
                do_sleep()
                continue
            else:
                for image in images:
                    src = image.find_element(By.TAG_NAME, 'img').get_attribute('src')
                    if src not in image_links:
                        image_links.append(src)
        else:
            for video in videos:
                src = video.get_attribute('src')
                if src not in video_links:
                    video_links.append(src)
        try:
            WebDriverWait(driver, 0.5).until(
                EC.presence_of_element_located(((By.XPATH, '//button[@aria-label="下一步"]')))
            ).click()    
        except Exception as e:
            break
        # do_sleep()
            
    return image_links, video_links

def download_image(url, path, name):
    if not os.path.exists(path):
        os.makedirs(path)
    file_format = "01"
    # Convert back to a string and use zfill to pad with zeros
    fullname = str(name).zfill(len(file_format))+".jpg"
    if not os.path.exists(path+fullname):
        urllib.request.urlretrieve(url,path+fullname)

def download_video(url, path, name):
    if not os.path.exists(path):
        os.makedirs(path)
    file_format = "01"
    # Convert back to a string and use zfill to pad with zeros
    fullname = str(name).zfill(len(file_format))+".mp4"
    if not os.path.exists(path+fullname):
        urllib.request.urlretrieve(url,path+fullname)
    
def download_post(driver, url, path):
    driver.get(url)
    # do_sleep()
    # find datetime
    while True:
        try:
            datetime = WebDriverWait(driver, 0.5).until(
                EC.presence_of_element_located(((By.TAG_NAME, "time")))
            ).get_attribute("datetime")
        except Exception as e:
            driver.refresh()
            continue
        else:
            datetime = remove_special_characters(datetime)
            break
    
    # find and download image
    image_links, video_links = find_images_videos(driver)
    print('datetime:', datetime, 'total image:', len(image_links), 'total video:', len(video_links))
    for count, image in enumerate(image_links):
        download_image(image, path+datetime+'/', count+1)
    for count, video in enumerate(video_links):
        download_video(video, path+datetime+'/', count+1)
        
def do_sleep():
    time.sleep(min(random.expovariate(0.8), 20.0))

if __name__ == "__main__":
    path = 'save_path'
    driver = opebwebdriver()
    login(driver, 'username', 'password')
    time.sleep(5)
    usernames = ['usernames']
    for username in usernames:
        links = find_all_post_link(driver, username)
        print('total post:', len(links))
        for link in links:
            download_post(driver, link, path)
    print('end')